In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Read and Load The Dataset

In [3]:
class DataLoader:
    def __init__(self,file_name):
        self.file_name = file_name
    def read_csv(self):
        df=pd.read_csv(self.file_name)
        return df
DataLoader_obj= DataLoader('cleaned_fintech_data.csv')
df = DataLoader_obj.read_csv()

In [4]:
df.sample(5)

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp
5026,2215.0,Thu Jun 17 09:56:02 +0000 2021,"<a href=""http://twitter.com/#!/download/ipad"" ...","LOL\n\nSwedish firm Lundin is selling ""carbon ...",LOLSwedish firm Lundin selling `` carbon neutr...,"Sentiment(polarity=-0.1, subjectivity=0.7)",-0.1,0.7,en,0,...,insaneEmoFemby,1499,39,286,NaN,NaN,DrSimEvans,NaN,"British Columbia, Canada",2021-06-17 09:56:02+00:00
2113,2113.0,Thu Jun 17 09:51:48 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",I don't want meat from Australia. Besides thei...,I n't want meat Australia Besides ghastly anim...,"Sentiment(polarity=0.0, subjectivity=0.3)",0.0,0.3,en,0,...,Lullora,19264,185,1041,NaN,NaN,artemis0801,NaN,london (uk),2021-06-17 09:51:48+00:00
4371,1560.0,Thu Jun 17 09:30:12 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","LOL\n\nSwedish firm Lundin is selling ""carbon ...",LOLSwedish firm Lundin selling `` carbon neutr...,"Sentiment(polarity=-0.1, subjectivity=0.7)",-0.1,0.7,en,0,...,LukeLochbaum,5070,46,1183,NaN,NaN,DrSimEvans,NaN,"Gettysburg, PA",2021-06-17 09:30:12+00:00
638,638.0,Thu Jun 17 08:49:51 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",#ThursdayTippaluTanthalu\n\nWhen you become a ...,become house worker rather husband marriage Tr...,"Sentiment(polarity=0.3, subjectivity=1.0)",0.3,1.0,en,0,...,NTR56574,2673,289,271,NaN,ThursdayTippaluTanthalu,PUsif4141,NaN,"Guntur, India",2021-06-17 08:49:51+00:00
4195,1384.0,Thu Jun 17 09:22:01 +0000 2021,"<a href=""http://twitter.com/download/android"" ...","Maa Hero Debute chesinappude ,mee Hero Debute ...",Maa Hero Debute chesinappude mee Hero Debute c...,"Sentiment(polarity=0.0, subjectivity=0.5)",0.0,0.5,tl,0,...,fgh11207329,49796,117,199,NaN,NaN,Varma_Tweetz,NaN,NaN,2021-06-17 09:22:01+00:00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5620 non-null   float64
 1   created_at              5621 non-null   object 
 2   source                  5621 non-null   object 
 3   original_text           5621 non-null   object 
 4   clean_text              5617 non-null   object 
 5   sentiment               5621 non-null   object 
 6   polarity                5621 non-null   object 
 7   subjectivity            5621 non-null   object 
 8   lang                    5621 non-null   object 
 9   favorite_count          5621 non-null   object 
 10  retweet_count           5621 non-null   object 
 11  original_author         5621 non-null   object 
 12  screen_count            5621 non-null   object 
 13  followers_count         5621 non-null   object 
 14  friends_count           5621 non-null   

Prepare The Data

In [13]:
from string import punctuation
class PrepareData:
    def __init__(self,df):
        self.df=df
    def prepare_data(self):
        df = self.df[self.df['lang'] =="en"]
        
        df['clean_text'] = df['clean_text'].fillna('')
        df['clean_text']=df['clean_text'].astype(str)
        df['clean_text'] = df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', punctuation)))
        df['clean_text'] = df['clean_text'].apply(lambda x: x.lower()) 
        
        
        #clean and prepare for feature engineering
        sentence_list = [sentence for sentence in df['clean_text']]
        vocab_list = [vocab.split() for vocab in sentence_list]

        return vocab_list
    def remove_stopwords(self,vocab_list):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in vocab_list]
    def make_bigrams(self,non_stop_words):
        return [bigram_mod[doc] for doc in non_stop_words]

    def make_trigrams(self,non_stop_words):
        return [trigram_mod[bigram_mod[doc]] for doc in non_stop_words]

    def lemmatization(self,make_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        texts_out = []
        for sent in make_bigrams:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    

In [15]:
data  = PrepareData(df)
vocab_list, corpus = data.prepare_data()
vocab_list

C:\Users\elleni fitsum\AppData\Local\Temp\ipykernel_21516\652485210.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].fillna('')
C:\Users\elleni fitsum\AppData\Local\Temp\ipykernel_21516\652485210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text']=df['clean_text'].astype(str)
C:\Users\elleni fitsum\AppData\Local\Temp\ipykernel_21516\652485210.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

[['giving',
  'forth',
  'life',
  'becoming',
  'burden',
  'kenya',
  'this',
  'mother',
  'gave',
  'birth',
  'babies',
  'hard',
  'time',
  'meeting',
  'nutritional',
  'needs',
  'coz',
  'baby',
  'formula',
  'expensiveturn',
  'bundles',
  'tears',
  'joy',
  'giving',
  'handmpesa',
  'douglas',
  'nyaoko'],
 ['today',
  'world',
  'day',
  'combat',
  'restoring',
  'degraded',
  'land',
  'brings',
  'jobs',
  'food',
  'security',
  'helps',
  'recover',
  'slows',
  'locking',
  'away',
  'carbon',
  'all',
  'critical',
  'lifting',
  'many',
  'poverty'],
 ['hearing',
  'say',
  's',
  'confident',
  'vaccines',
  'delivered',
  'like',
  'hearing',
  'say',
  'g7',
  's',
  'reducing',
  'carbon',
  'emissions'],
 ['buy',
  'airtime',
  'across',
  'networks',
  'mpesa',
  'even',
  'fuliza',
  'via',
  'paybill',
  'account',
  'number',
  'phone',
  'number',
  'kabando',
  'locatelli',
  'ramos',
  'arteta',
  'tony'],
 ['today',
  'world',
  'day',
  'combat',
 

In [21]:
bigram = gensim.models.Phrases(vocab_list, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[vocab_list], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[vocab_list[0]]])

['giving', 'forth', 'life', 'becoming', 'burden', 'kenya', 'this', 'mother', 'gave', 'birth', 'babies', 'hard', 'time', 'meeting', 'nutritional', 'needs', 'coz', 'baby', 'formula', 'expensiveturn', 'bundles', 'tears', 'joy', 'giving', 'handmpesa', 'douglas', 'nyaoko']


In [28]:
python -m spacy download en

  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-04-27 22:05:21.313042: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll


In [31]:
nlp = spacy.load('en_core_web_sm')
non_stop_words = data.remove_stopwords(vocab_list)
make_bigrams = data.make_bigrams(non_stop_words)
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = data.lemmatization(make_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['give', 'life', 'become', 'burden', 'kenya', 'mother', 'give', 'birth', 'baby', 'hard', 'time', 'meet', 'nutritional', 'need', 'coz', 'baby', 'formula', 'expensiveturn', 'bundle', 'tear', 'joy', 'give']]


In [33]:
#create a dictionary
vocab_to_int = corpora.Dictionary(vocab_list)
corpus= [vocab_to_int.doc2bow(vocab) for vocab in vocab_list]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]]


In [35]:
[[(vocab_to_int[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('babies', 1),
  ('baby', 1),
  ('becoming', 1),
  ('birth', 1),
  ('bundles', 1),
  ('burden', 1),
  ('coz', 1),
  ('douglas', 1),
  ('expensiveturn', 1),
  ('formula', 1),
  ('forth', 1),
  ('gave', 1),
  ('giving', 2),
  ('handmpesa', 1),
  ('hard', 1),
  ('joy', 1),
  ('kenya', 1),
  ('life', 1),
  ('meeting', 1),
  ('mother', 1),
  ('needs', 1),
  ('nutritional', 1),
  ('nyaoko', 1),
  ('tears', 1),
  ('this', 1),
  ('time', 1)]]

In [37]:
#Building The Topic Model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=vocab_to_int,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [40]:
#let's View the Topics 
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.154*"gt" + 0.049*"rc" + 0.047*"office" + 0.047*"mb" + 0.047*"box" + '
  '0.028*"kuda" + 0.027*"fanbase" + 0.027*"records" + 0.027*"ok" + '
  '0.026*"gurinchi"'),
 (1,
  '0.135*"carbon" + 0.118*"amp" + 0.074*"it" + 0.069*"would" + 0.067*"giving" '
  '+ 0.056*"this" + 0.052*"we" + 0.051*"using" + 0.050*"people" + '
  '0.037*"find"'),
 (2,
  '0.195*"i" + 0.112*"need" + 0.067*"pushed" + 0.046*"live" + 0.043*"low" + '
  '0.042*"get" + 0.040*"transport" + 0.037*"run" + 0.031*"carbon" + '
  '0.023*"power"'),
 (3,
  '0.142*"carbon" + 0.117*"pm" + 0.096*"go" + 0.072*"join" + 0.057*"webinar" + '
  '0.055*"today" + 0.054*"worlds" + 0.048*"become" + 0.024*"amount" + '
  '0.023*"lowcarbon"'),
 (4,
  '0.097*"one" + 0.067*"global" + 0.050*"emissions" + 0.046*"if" + 0.044*"s" + '
  '0.034*"day1" + 0.031*"kuda" + 0.031*"largest" + 0.030*"day" + '
  '0.029*"impact"'),
 (5,
  '0.122*"absolutely" + 0.096*"support" + 0.045*"team" + 0.040*"play" + '
  '0.030*"class" + 0.029*"carbon" + 0.029*"alway

In [43]:
#Evaluation of the Model 
# Perplexity
print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=vocab_to_int, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Perplexity:  -14.89645618874194
Coherence Score:  nan


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis